# OpenAI Assistants APIs

The Assistants' API lets you create AI assistants in your applications. These assistants follow instructions and use models, tools, and knowledge to answer user questions. In this notebook we are going to use one of the tools, retriever,
to query against two pdf documents we will upload.

The architeture and data flow diagram below depicts the interaction among all
components that comprise OpenAI Assistant APIs. Central to understand is the 
Threads and Runtime that executes anyschronously, adding and reading messages
to the Threads.

For integrating the Assistants API:

1. Creat an Assistant with custom instructions and select a model. Optionally, enable tools like Code Interpreter, Retrieval, and Function Calling.

2. Initiate a Thread for each user conversation.
    
3. Add user queries as Messages to the Thread.

4.  Run the Assistant on the Thread for responses, which automatically utilizes the enabled tools

Below we follow those steps to demonstrate how to integrate Assistants API,
usijng Retrieval tool, to a) upload a couple of pdf documents and b) use Assistant
to query the contents of the document. Consider this as a mini Retrieval Augmented Generation (RAG). 

The OpenAI documentation describes in details [how Assistants work](https://platform.openai.com/docs/assistants/how-it-works).

<img src="./images/assistant_ai_tools_retriever.png">


## How to use Assistant API using Tools: Retriever

In [1]:
import warnings
import os

import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
from typing import List

Load our .env file with respective API keys and base url endpoints. Here you can either use OpenAI or Anyscale Endpoints. **Note**: Assistant API calling for Anyscale Endpoints (which serves only OS modles) is not yet aviable).

In [2]:
warnings.filterwarnings('ignore')

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))
MODEL = os.getenv("MODEL")
print(f"Using MODEL={MODEL}; base={openai.api_base}")

Using MODEL=gpt-4-1106-preview; base=https://api.openai.com/v1


In [3]:
DOCS_TO_LOAD = ["docs/HAI_AI-Index-Report_2023.pdf"]

Utility function to create OpenAI client instance

In [4]:
from openai import OpenAI

client = OpenAI(
    api_key = openai.api_key,
    base_url = openai.api_base
)

In [5]:
from pprint import pprint
def print_messages(clnt: object, thrd: object) -> None:
    messages = clnt.beta.threads.messages.list(
        thread_id = thrd.id)
    for msg in messages:
        pprint(msg.role + ":" + msg.content[0].text.value)

In [6]:
def upload_files(clnt: object, docs: List[str]) -> List[object]:
    file_objs = []
    for doc in docs:
        print(f"Uploading doc:{doc}...")
        file_obj = clnt.files.create(file=open(doc, "rb"), 
                                    purpose="assistants")
        file_objs.append(file_obj)
    return file_objs

In [7]:
import time
def loop_until_completed(clnt: object, thrd: object, run_obj: object) -> None:
    while run_obj.status not in ["completed", "failed"]:
        run_obj = clnt.beta.threads.runs.retrieve(
            thread_id = thrd.id,
            run_id = run_obj.id)
        time.sleep(10)
        print(run_obj.status)

In [8]:
def create_assistant_run(clnt: object, asst: object, thrd: object) -> object:
    run = clnt.beta.threads.runs.create(
    thread_id=thrd.id,
    assistant_id=asst.id,
    instructions = """Please address the user as Jules Dmatrix.  
    Do not provide an answer to the question if the information was not retrieved from the knowledge base.
"""
)
    return run
    

### Step 1: Create our knowledgebase
This entails uploading your pdfs as your knowledgebase for the retrievers to use.
Most likely, these docuements will be chuncked and indexed as vectors into a vector store; that bit is hidden from you.

The retireivers use your query to search retrieve the best semantic matches on vectors in the knowledgebase, and then feed the LLM, along with the original query, to generate the consolidated and comprehesive answer, similarly to how
a RAG retriever operates.

In [9]:
file_objects = upload_files(client, DOCS_TO_LOAD)
file_objects

Uploading doc:docs/HAI_AI-Index-Report_2023.pdf...


[FileObject(id='file-vAQcWhQsQXt0j1xwuxYnokKr', bytes=25318310, created_at=1703095478, filename='HAI_AI-Index-Report_2023.pdf', object='file', purpose='assistants', status='processed', status_details=None)]

In [10]:
# Extract file ids 
file_obj_ids = []
for f_obj in file_objects:
    file_obj_ids.append(file_objects[0].id)
file_obj_ids

['file-vAQcWhQsQXt0j1xwuxYnokKr']

### Step 2: Create an Assistant 
Before you can start interacting with the Assistant to carry out any tasks, you need an AI assistant object.

Provide the Assistant with a model to use, tools, and file ids to use for its
knowledge base.

In [11]:
assistant = client.beta.assistants.create(name="AI Report and LLM survey Chatbot",
                                           instructions="""You are a knowledgeable chatbot trained to respond 
                                               inquires on documents HAI Artificial Index 2023 report 
                                               and Survey of why LLMs hallucinate. 
                                               Use a neutral, professional advisory tone, and only respond by consulting the 
                                               knowledge base or files you are granted access to. 
                                               Do not make up answers. If you don't know answer, respond with 'Sorry, I'm afraid
                                               I don't have access to that information.'""",
                                           model=MODEL,
                                           tools = [{'type': 'retrieval'}],  # use the retrieval tool
                                           file_ids=file_obj_ids # use these files uploaded as part of your knowledge base
)                                        
assistant

Assistant(id='asst_22BVY7nCoy2XspFAMFSrT9Pl', created_at=1703095479, description=None, file_ids=['file-vAQcWhQsQXt0j1xwuxYnokKr'], instructions="You are a knowledgeable chatbot trained to respond \n                                               inquires on documents HAI Artificial Index 2023 report \n                                               and Survey of why LLMs hallucinate. \n                                               Use a neutral, professional advisory tone, and only respond by consulting the \n                                               knowledge base or files you are granted access to. \n                                               Do not make up answers. If you don't know answer, respond with 'Sorry, I'm afraid\n                                               I don't have access to that information.'", metadata={}, model='gpt-4-1106-preview', name='AI Report and LLM survey Chatbot', object='assistant', tools=[ToolRetrieval(type='retrieval')])

### Step 3: Create a thread 
As the diagram above shows, the Thread is the object with which the AI Assistant runs will interact with, by fetching messages and putting messages to it. Think of a thread as a "conversation session between an Assistant and a user. Threads store Messages and automatically handle truncation to fit content into a model’s context window."

In [12]:
thread = client.beta.threads.create()
thread

Thread(id='thread_h60KCywOk4mHeVdDK0JvwvQk', created_at=1703095488, metadata={}, object='thread')

### Step 4: Add your message query to the thread for the Assistant

In [13]:
message_1 = client.beta.threads.messages.create(
    thread_id=thread.id, 
    role="user",
    content="""What are the top 10 takeaways in the Artificial Intelligence Index Report 2023.
    Summarize each takeway in no more three simple sentences.""",
)
message_1

ThreadMessage(id='msg_N7zA7p12HE8x4ElARW8xlDQn', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What are the top 10 takeaways in the Artificial Intelligence Index Report 2023.\n    Summarize each takeway in no more three simple sentences.'), type='text')], created_at=1703095489, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_h60KCywOk4mHeVdDK0JvwvQk')

### Step 5: Create a Run for the Assistant
A Run is an invocation of an Assistant on a Thread. The Assistant uses it’s configuration and the Thread’s Messages to perform tasks by calling models and tools. As part of a Run, the Assistant appends Messages to the Thread.

Note that Assistance will run asychronously: the run has the following
lifecycle and states: [*expired, completed, failed, cancelled*]. Run objects can have multiple statuses.

<img src="https://cdn.openai.com/API/docs/images/diagram-1.png">

In [14]:
run_1 = create_assistant_run(client, assistant, thread)
run_1

Run(id='run_0IGlWSfETHkAVMPhbWPoVKIB', assistant_id='asst_22BVY7nCoy2XspFAMFSrT9Pl', cancelled_at=None, completed_at=None, created_at=1703095489, expires_at=1703096089, failed_at=None, file_ids=['file-vAQcWhQsQXt0j1xwuxYnokKr'], instructions='Please address the user as Jules Dmatrix.  \n    Do not provide an answer to the question if the information was not retrieved from the knowledge base.\n', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_h60KCywOk4mHeVdDK0JvwvQk', tools=[ToolAssistantToolsRetrieval(type='retrieval')])

### Step 6: Retrieve the status 

In [15]:
run_1_obj = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run_1.id
)

print(run_1_obj.status)

in_progress


In [16]:
loop_until_completed(client, thread, run_1_obj)

in_progress
in_progress
in_progress
in_progress
completed


### Step 7: Retrieve the message returned by the assistance
Only when the run is **completed** can you fetch the messages from the Thread

In [17]:
print_messages(client, thread)

('assistant:Here are the top 10 takeaways from the Artificial Intelligence '
 'Index Report 2023, each summarized in up to three sentences:\n'
 '\n'
 '1. **Industry Surpasses Academia**: Since 2014, the production of '
 'significant machine learning models has shifted from academia to industry. '
 'In 2022, there were 32 significant industry-produced models versus only 3 '
 'from academia, as industry inherently has more resources like data, computer '
 'power, and money.\n'
 '\n'
 '2. **Benchmark Saturation**: AI continues to achieve state-of-the-art '
 'results, but gains on many benchmarks are marginal and saturating quickly. '
 'New, more complex benchmarks like BIG-bench and HELM are being introduced.\n'
 '\n'
 '3. **Environmental Impact**: AI can significantly affect the environment, '
 'with training for models like BLOOM emitting large amounts of carbon. '
 'However, AI can also optimize energy usage, as shown by models like '
 'BCOOLER.\n'
 '\n'
 '4. **AI Accelerating Science*

### Repeat the process for any additional messages
To add more query messages to the thread for the Assistant,
repeat steps 5 - 7

### Add another message to for the Assistant

In [18]:
message_2 = client.beta.threads.messages.create(
    thread_id=thread.id, 
    role="user",
    content="""Provide a short overview of Chatper 8 on public opinion in no more than
    five sentences
    """,
)
message_2

ThreadMessage(id='msg_YVtEXNHeE2Tqy1YiQaiCG2I9', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Provide a short overview of Chatper 8 on public opinion in no more than\n    five sentences\n    '), type='text')], created_at=1703095541, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_h60KCywOk4mHeVdDK0JvwvQk')

### Create another run for the Assistant for the second message

In [19]:
run_2 = create_assistant_run(client, assistant, thread)
run_2

Run(id='run_T3rsEYQwh08XJ4zlW3kDgkhX', assistant_id='asst_22BVY7nCoy2XspFAMFSrT9Pl', cancelled_at=None, completed_at=None, created_at=1703095541, expires_at=1703096141, failed_at=None, file_ids=['file-vAQcWhQsQXt0j1xwuxYnokKr'], instructions='Please address the user as Jules Dmatrix.  \n    Do not provide an answer to the question if the information was not retrieved from the knowledge base.\n', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_h60KCywOk4mHeVdDK0JvwvQk', tools=[ToolAssistantToolsRetrieval(type='retrieval')])

In [20]:
run_2_obj = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run_2.id
)

print(run_2_obj.status)

in_progress


In [21]:
loop_until_completed(client, thread, run_2_obj)

in_progress
in_progress
in_progress
in_progress
completed


In [22]:
print_messages(client, thread)

('assistant:Chapter 8 of the Artificial Intelligence Index Report 2023 focuses '
 'on the importance of monitoring public opinion regarding AI due to its '
 'potential to significantly impact society. Understanding these opinions '
 "helps to inform decisions related to AI's development, regulation, and "
 'application. The chapter discusses public perception from global, national, '
 'demographic, ethnic, and AI researcher perspectives and concludes by '
 'examining social media discourse about AI in 2022. It utilizes data from '
 "IPSOS, Lloyd's Register Foundation and Gallup, as well as PEW Research. The "
 'chapter notes a lack of extensive longitudinal data on AI opinions, '
 'emphasizing the need for greater efforts to comprehend these views as AI '
 'becomes increasingly prevalent【13†source】.')
('user:Provide a short overview of Chatper 8 on public opinion in no more '
 'than\n'
 '    five sentences\n'
 '    ')
('assistant:Here are the top 10 takeaways from the Artificial Intell

In [23]:
# Delete the assistant. Optionally, you can delete any files
# associated with it that you have uploaded onto the OpenAI platform

response = client.beta.assistants.delete(assistant.id)
print(response)

for file_id in file_obj_ids:
    print(f"deleting file id: {file_id}...")
    response = client.files.delete(file_id)
    print(response)

AssistantDeleted(id='asst_22BVY7nCoy2XspFAMFSrT9Pl', deleted=True, object='assistant.deleted')
deleting file id: file-vAQcWhQsQXt0j1xwuxYnokKr...
FileDeleted(id='file-vAQcWhQsQXt0j1xwuxYnokKr', deleted=True, object='file')
